In [1]:
#Sathya Sravya  20161121
#FINAL RESULTS OBTAINED 

#pca comps = 100 to 110
#lda comps = 350
#num of trees = 460
#max depth = 420 to 450
#min tree split = 14 


#number of neurons in first layer = max possible 5000
#num of pca comps = 100
#activation functions order : relu logistic tanh identity



## NOTE 

Here functions with main suffix run that respective classifier on three data types and functions with prefixes vary can be commented and run .You can call those functions if you want to vary those parameters else comment them out in the last cell (26th) (heading : caller to all functions)

And the function names are self explanatory !

In [2]:
import os
import cv2
import pickle
import numpy as np
import pdb
import requests
from collections import defaultdict
import random 
import time
import matplotlib.pyplot as plt 

from sklearn import preprocessing
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
#import snips as snp
from sklearn.metrics import f1_score, accuracy_score

from tqdm import *

from functools import wraps
from time import time as _timenow 
from sys import stderr


## Load CIFAR-10 Data

In [3]:
def load_cifar():
    
    trn_data, trn_labels, tst_data, tst_labels = [], [], [], []
    def unpickle(file):
        with open(file, 'rb') as fo:
            data = pickle.load(fo, encoding='latin1')
        return data
    
    for i in trange(1):
        batchName = './data/data_batch_{0}'.format(i + 1)
        unpickled = unpickle(batchName)
        trn_data.extend(unpickled['data'])
        trn_labels.extend(unpickled['labels'])
    unpickled = unpickle('./data/test_batch')
    tst_data.extend(unpickled['data'])
    tst_labels.extend(unpickled['labels'])
    return trn_data, trn_labels, tst_data, tst_labels


## Image preprocessing

In [4]:
def image_prep(train_data,test_data):
    ''' pre-processes the given image
        performs mean normalization and other such operations http://scikit-learn.org/stable/modules/preprocessing.html'''
    #feature scaling
    scaler = preprocessing.StandardScaler().fit(train_data)
    train_processed = scaler.transform(train_data)
    test_processed = scaler.transform(test_data)
    
    return train_processed,test_processed

## Dimensionality reduction using PCA

In [5]:
def reduce_dim(**kwargs):
    ''' performs dimensionality reduction
    https://stackabuse.com/implementing-lda-in-python-with-scikit-learn/'''
    if kwargs['method'] == 'pca':
        pca = PCA(n_components=kwargs['num_comp'])
        pca.whiten = True
        pca = pca.fit(kwargs['train_data'])
        train_reduced = pca.transform(kwargs['train_data'])
        test_reduced = pca.transform(kwargs['test_data'])
        return train_reduced,test_reduced
    if kwargs['method'] == 'lda':
        lda = LDA(n_components=kwargs['num_comp'])
        lda = lda.fit(kwargs['train_data'],kwargs['train_label'])
        train_reduced = lda.transform(kwargs['train_data'])
        test_reduced = lda.transform(kwargs['test_data'])
        return train_reduced,test_reduced
        

## Classification using decision tree,SVM(rbf),MLP and LR

In [6]:
def classify(X, y, **kwargs):
    ''' trains a classifier by taking input features
        and their respective targets and returns the trained model'''
    if kwargs['method'] == 'CART':
        clf = RandomForestClassifier(n_estimators=kwargs['num_trees'], min_samples_split=kwargs['min_samples_split'], max_depth =kwargs['max_depth_tree'],bootstrap=kwargs['bootstr'])
        clf.fit(X,y)
        return clf
    if kwargs['method'] == 'SVM':
        clf_svm = SVC(kernel=kwargs['choice_kernel'],
                    max_iter=kwargs['num_iter'],C=kwargs['c'],
                    degree=kwargs['degree'])
        clf_svm.fit(X,y)
        return clf_svm
    if kwargs['method']=='MLP':
        mlp = MLPClassifier(hidden_layer_sizes=(kwargs['h1']
                                                , ),activation= kwargs['activationfn']                  
                    )#,activation , max_iter=200 default (early_stopping boolfalse default)solver=kwargs['sgd'],
        #mlp.activation = 
        mlp.fit(X, y)
        #snp.prettyplot(matplotlib)
        #fig, ax = snp.newfig()
        #ax.plot(classifier.loss_curve_)
        #snp.labs("number of steps", "loss function", "Loss During GD (Rate=0.001)")
       # print("Training set score: %f" % mlp.score(X, y))
        return mlp
    if kwargs['method']=='LogisticRegression':
        lr = LogisticRegression(penalty=kwargs['penalty'],
                    max_iter=kwargs['num_iter'],C=kwargs['c'])
        lr.fit(X,y)
        return lr
        

## Evaluation 

In [7]:
def evaluate(target, predicted):
    f1 = f1_score(target, predicted, average='micro')
    acc = accuracy_score(target, predicted)
    return f1, acc

In [8]:
def test(**kwargs):
    '''takes test data and trained classifier model,
    performs classification and prints accuracy and f1-score'''
    if(kwargs['method']=='CART'):
        model = kwargs['model']
        X_test = kwargs['test']
        predicted = model.predict(X_test)     
        return predicted
    if(kwargs['method']=='MLP'):
        model = kwargs['model']
        y_test = kwargs['test_label']
        X_test = kwargs['test']
        predicted = model.predict(X_test)
        print("Test set score: %f" % model.score(X_test, y_test))
        return predicted
    else:
        model = kwargs['model']
        y_test = kwargs['test_label']
        X_test = kwargs['test']
        predicted = model.predict(X_test)
        return predicted

## Function 'main' for all tree classifier

In [9]:
def main_tree():
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    #trn_data, tst_data = image_prep(trn_data, tst_data)#, list(map(image_prep, tst_data))
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    ''' perform dimesioality reduction/feature extraction and classify the features into one of 10 classses
        print accuracy and f1-score.
    https://www.geeksforgeeks.org/args-kwargs-python/ #kwargs usage
        '''
    trprep_data, teprep_data = image_prep(X_train, X_val) 
        
    tr_data1,te_data1 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=100)
    tr_data2,te_data2 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='lda',num_comp=350)
    
    model = classify(tr_data1, y_train, num_trees=600, min_samples_split=4, max_depth_tree=470,bootstr =True,method='CART')
    modelmlp = classify(tr_data1, y_train,method='MLP',
                        h1=5000,activationfn='logistic')
    modellr = classify(tr_data1,y_train,method='LogisticRegression',penalty='l2',num_iter=150,c=1.0)
    
    labels_predicted = test(test=te_data1,test_label=y_val,model=model,method='CART')
    #labels_predicted = test(test=te_data1,test_label=tst_labels,model=modellr,method='LogisticRegression')
    f_score, accuracy = evaluate(y_val, labels_predicted)
    print('Val-PCA reduced data - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
    
    model = classify(trprep_data, y_train, num_trees=600, min_samples_split=4, max_depth_tree=470,bootstr =True,method='CART')
    labels_predicted = test(test=teprep_data,test_label=y_val,model=model,method='CART')
    #labels_predicted = test(test=te_data1,test_label=tst_labels,model=modellr,method='LogisticRegression')
    f_score, accuracy = evaluate(y_val, labels_predicted)
    print('Val-RAW data - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
    
    model = classify(tr_data2, y_train, num_trees=600, min_samples_split=4, max_depth_tree=470,bootstr =True,method='CART')
    labels_predicted = test(test=te_data2,test_label=y_val,model=model,method='CART')
    #labels_predicted = test(test=te_data1,test_label=tst_labels,model=modellr,method='LogisticRegression')
    f_score, accuracy = evaluate(y_val, labels_predicted)
    print('Val-LDA reduced data - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
    
    
    
#main_tree()    

## Parameter tuning for Decision trees 

In [10]:
def vary_num_trees():#trprep_data,teprep_data,y_train,y_test): 
    num_trees_arr=[]
    acc=[]
    i=200
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
   # trn_data, tst_data = image_prep(trn_data, tst_data)#, list(map(image_prep, tst_data))
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    ''' perform dimesioality reduction/feature extraction and classify the features into one of 10 classses
        print accuracy and f1-score.
    https://www.geeksforgeeks.org/args-kwargs-python/ #kwargs usage
        '''
    trprep_data, teprep_data = image_prep(X_train, X_val)
    trprep_data,teprep_data = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=100)
    while(i<620):
        #train_data, test_data = image_prep(X_train, X_test)
        model = classify(trprep_data, y_train, num_trees=i, min_samples_split=4,max_depth_tree=470,bootstr =True, method='CART')
        output = test(test = teprep_data,test_label=y_val, model= model, method='CART')
        f_score, accuracy = evaluate(y_val, output)
        print('Val - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
        acc.append(accuracy)
        num_trees_arr.append(i)
        
        i+=50
        
    plt.plot(num_trees_arr, acc)
    plt.xlabel('No. of trees used') 
    plt.ylabel('Accuracy') 
    plt.show()
#vary_num_trees()

In [11]:
def vary_num_pca(): #using randomforestclassifer
    num_comp_pca=[]
    acc=[]
    i=50
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
   # trn_data, tst_data = image_prep(trn_data, tst_data)#, list(map(image_prep, tst_data))
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    trprep_data, teprep_data = image_prep(X_train, X_val)
    while(i<360):
        tr_data1,te_data1 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=i)
        model = classify(tr_data1, y_train, num_trees=600, min_samples_split=4,max_depth_tree=470,bootstr =True, method='CART')
        output = test(test = te_data1, test_label=y_val,model= model, method='CART')
        f_score, accuracy = evaluate(y_val, output)
        print('Val - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
        acc.append(accuracy)
        num_comp_pca.append(i)
        i+=50
    
    plt.plot(num_comp_pca, acc)
    plt.xlabel('No. of pca components used') 
    plt.ylabel('Accuracy') 
    plt.show()
    
    
    
    

In [12]:
def vary_num_lda(): #using randomforestclassifer
    num_comp_lda=[]
    acc=[]
    i=50
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
   # trn_data, tst_data = image_prep(trn_data, tst_data)#, list(map(image_prep, tst_data))
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    trprep_data, teprep_data = image_prep(X_train, X_val)
    while(i<800):
        tr_data1,te_data1 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='lda',num_comp=i)
        model = classify(tr_data1, y_train, num_trees=600, min_samples_split=4,max_depth_tree=470,bootstr =True, method='CART')
        output = test(test = te_data1, model= model, method='CART')
        f_score, accuracy = evaluate(y_val, output)
        print('Val - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
        acc.append(accuracy)
        num_comp_lda.append(i)
        i+=50
    
    plt.plot(num_comp_lda, acc)
    plt.xlabel('No. of LDA components used') 
    plt.ylabel('Accuracy') 
    plt.show()
    

In [13]:
def vary_min_split_tree():
    min_split_arr=[]
    acc=[]
    i=2
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    #trn_data, tst_data = image_prep(trn_data, tst_data)#, list(map(image_prep, tst_data))
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    trprep_data, teprep_data = image_prep(X_train, X_val)
    trprep_data,teprep_data = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=100)
    while(i<17):
        model = classify(trprep_data, y_train, num_trees=570, min_samples_split=i,max_depth_tree=455,bootstr =True, method='CART')
        output = test(test = teprep_data, model= model, method='CART')
        f_score, accuracy = evaluate(y_val, output)
        print('Val - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
        acc.append(accuracy)
        min_split_arr.append(i)
        
        i+=2
        
    plt.plot(min_split_arr, acc)
    plt.xlabel('No. of splits minimum used') 
    plt.ylabel('Accuracy') 
    plt.show()
    
#vary_min_split_tree()

In [14]:
def vary_depth_tree():
    max_depth_arr=[]
    acc=[]
    i=50
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    #trn_data, tst_data = image_prep(trn_data, tst_data)#, list(map(image_prep, tst_data))
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    trprep_data, teprep_data = image_prep(X_train, X_val)
    trprep_data,teprep_data = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=100)
    while(i<990):
        model = classify(trprep_data, y_train, num_trees=300, min_samples_split=5,max_depth_tree=i,bootstr =True, method='CART')
        output = test(test = teprep_data, model= model, method='CART')
        f_score, accuracy = evaluate(y_val, output)
        print('Val - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
        acc.append(accuracy)
        max_depth_arr.append(i)
        
        i+=100
        
    plt.plot(max_depth_arr, acc)
    plt.xlabel('Max depth that can be used') 
    plt.ylabel('Accuracy') 
    plt.show()
    
#vary_depth_tree()

## SVM (rbf kernel) main call and tuning

In [15]:
def main_svm():
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    ''' perform dimesioality reduction/feature extraction and classify the features into one of 10 classses
        print accuracy and f1-score.
    https://www.geeksforgeeks.org/args-kwargs-python/ #kwargs usage
        '''
    trprep_data, teprep_data = image_prep(X_train, X_val) 
        
    tr_data1,te_data1 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=100)
    tr_data2,te_data2 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='lda',num_comp=350)
    modelsvmrbf = classify(tr_data1,y_train,method='SVM',choice_kernel='rbf',num_iter=150,c=1.0,degree=3)
    
    labels_predicted = test(test=te_data1,test_label=y_val,model=modelsvmrbf,method='SVM')
    #labels_predicted = test(test=te_data1,test_label=tst_labels,model=modellr,method='LogisticRegression')
    f_score, accuracy = evaluate(y_val, labels_predicted)
    print('Val-PCA reduced data -SVM - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
    
    modelsvmrbf = classify(trprep_data,y_train,method='SVM',choice_kernel='rbf',num_iter=150,c=1.0,degree=3)
    
    labels_predicted = test(test=teprep_data,test_label=y_val,model=modelsvmrbf,method='SVM')
    #labels_predicted = test(test=te_data1,test_label=tst_labels,model=modellr,method='LogisticRegression')
    f_score, accuracy = evaluate(y_val, labels_predicted)
    print('Val-RAW data -SVM - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
    
    modelsvmrbf = classify(tr_data2,y_train,method='SVM',choice_kernel='rbf',num_iter=150,c=1.0,degree=3)
    
    labels_predicted = test(test=te_data2,test_label=y_val,model=modelsvmrbf,method='SVM')
    #labels_predicted = test(test=te_data1,test_label=tst_labels,model=modellr,method='LogisticRegression')
    f_score, accuracy = evaluate(y_val, labels_predicted)
    print('Val-LDA reduced data -SVM - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
    
    
    

In [16]:
def vary_ker_svm():
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    ''' perform dimesioality reduction/feature extraction and classify the features into one of 10 classses
        print accuracy and f1-score.
    https://www.geeksforgeeks.org/args-kwargs-python/ #kwargs usage
        '''
    trprep_data, teprep_data = image_prep(X_train, X_val) 
    str = ['linear','poly','rbf','sigmoid']    
    res_arr=[]
    acc = []
    tr_data1,te_data1 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=100)
    i =0
    while(i<4):
    
        modelsvm = classify(tr_data1,y_train,method='SVM',choice_kernel=str[i],num_iter=150,c=1.0,degree=3)
    
        labels_predicted = test(test=te_data1,test_label=y_val,model=modelsvm,method='SVM')
        #labels_predicted = test(test=te_data1,test_label=tst_labels,model=modellr,method='LogisticRegression')
        f_score, accuracy = evaluate(y_val, labels_predicted)
        print('Val-PCA reduced data -SVM - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
        acc.append(accuracy)
        res_arr.append(i)
        
        i+=1
        
        
    plt.plot(res_arr, acc)
    plt.xlabel('Max depth that can be used') 
    plt.ylabel('Accuracy') 
    plt.show()
    
#vary_ker_svm()

In [17]:
def vary_num_iter_svm():
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    ''' perform dimesioality reduction/feature extraction and classify the features into one of 10 classses
        print accuracy and f1-score.
    https://www.geeksforgeeks.org/args-kwargs-python/ #kwargs usage
        '''
    trprep_data, teprep_data = image_prep(X_train, X_val) 
    res_arr=[]
    acc = []
    tr_data1,te_data1 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=100)
    i =150
    while(i<6200):
    
        modelsvm = classify(tr_data1,y_train,method='SVM',choice_kernel='rbf',num_iter=i,c=1.0,degree=3)
    
        labels_predicted = test(test=te_data1,test_label=y_val,model=modelsvm,method='SVM')
        #labels_predicted = test(test=te_data1,test_label=tst_labels,model=modellr,method='LogisticRegression')
        f_score, accuracy = evaluate(y_val, labels_predicted)
        print('Val-PCA reduced data -SVM - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
        acc.append(accuracy)
        res_arr.append(i)
        
        i+=1000
        
        
    plt.plot(res_arr, acc)
    plt.xlabel('Max number of iterations that can be used') 
    plt.ylabel('Accuracy') 
    plt.show()
    

In [18]:
def vary_pca_svm():
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    ''' perform dimesioality reduction/feature extraction and classify the features into one of 10 classses
        print accuracy and f1-score.
    https://www.geeksforgeeks.org/args-kwargs-python/ #kwargs usage
        '''
    trprep_data, teprep_data = image_prep(X_train, X_val) 
    res_arr=[]
    acc = []
    
    i =150
    while(i<1200):
        tr_data1,te_data1 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=i)
        modelsvm = classify(tr_data1,y_train,method='SVM',choice_kernel='rbf',num_iter=1200,c=1.0,degree=3)
    
        labels_predicted = test(test=te_data1,test_label=y_val,model=modelsvm,method='SVM')
        #labels_predicted = test(test=te_data1,test_label=tst_labels,model=modellr,method='LogisticRegression')
        f_score, accuracy = evaluate(y_val, labels_predicted)
        print('Val-PCA reduced data -SVM - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
        acc.append(accuracy)
        res_arr.append(i)
        
        i+=100
        
        
    plt.plot(res_arr, acc)
    plt.xlabel('Max number pca components that can be used') 
    plt.ylabel('Accuracy') 
    plt.show()

## Main call for classifying 3 data types and Parameter tuning For MLP 

In [19]:
def main_mlp():
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    #trn_data, tst_data = image_prep(trn_data, tst_data)#, list(map(image_prep, tst_data))
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    ''' perform dimesioality reduction/feature extraction and classify the features into one of 10 classses
        print accuracy and f1-score.
    https://www.geeksforgeeks.org/args-kwargs-python/ #kwargs usage
        '''
    trprep_data, teprep_data = image_prep(X_train, X_val) 
        
    tr_data1,te_data1 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=100)
    tr_data2,te_data2 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='lda',num_comp=350)
    
    modelmlp = classify(tr_data1, y_train,method='MLP',
                        h1=5000,activationfn='relu')
    labels_predicted = test(test=te_data1,test_label=y_val,model=modelmlp,method='MLP')
    #labels_predicted = test(test=te_data1,test_label=tst_labels,model=modellr,method='LogisticRegression')
    f_score, accuracy = evaluate(y_val, labels_predicted)
    print('Val-PCA reduced data MLP - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
    
    modelmlp = classify(trprep_data, y_train,method='MLP',
                        h1=5000,activationfn='relu')
    labels_predicted = test(test=teprep_data,test_label=y_val,model=modelmlp,method='MLP')
    #labels_predicted = test(test=te_data1,test_label=tst_labels,model=modellr,method='LogisticRegression')
    f_score, accuracy = evaluate(y_val, labels_predicted)
    print('Val-RAW data MLP - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
    
    modelmlp = classify(tr_data2, y_train,method='MLP',
                        h1=5000,activationfn='relu')
    labels_predicted = test(test=te_data2,test_label=y_val,model=modelmlp,method='MLP')
    #labels_predicted = test(test=te_data1,test_label=tst_labels,model=modellr,method='LogisticRegression')
    f_score, accuracy = evaluate(y_val, labels_predicted)
    print('Val-LDA reduced data MLP - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
    
    

In [20]:
def num_neurons_l1():
    num_neurons_l1_arr=[]
    acc=[]
    i=3000
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    #trn_data, tst_data = image_prep(trn_data, tst_data)#, list(map(image_prep, tst_data))
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    trprep_data, teprep_data = image_prep(X_train, X_val)
    tr_data1,te_data1 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=100)
    while(i<5002):
        modelmlp = classify(tr_data1, y_train,method='MLP',
                        h1=i,activationfn='logistic')
    
    
        labels_predicted = test(test=te_data1,test_label=y_val,model=modelmlp,method='MLP')
        f_score, accuracy = evaluate(y_val, labels_predicted)
     
    
        print('Val - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
        acc.append(accuracy)
        num_neurons_l1_arr.append(i)
        
        i+=1000
        
    plt.plot(num_neurons_l1_arr, acc)
    plt.xlabel('Max neurons that can be used') 
    plt.ylabel('Accuracy') 
    plt.show()
        
    

In [21]:
def vary_activ_fn():
    str=['logistic','relu','tanh','identity']
    actfn_index_arr=[]
    acc=[]
    i=0
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    #trn_data, tst_data = image_prep(trn_data, tst_data)#, list(map(image_prep, tst_data))
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    trprep_data, teprep_data = image_prep(X_train, X_val)
    tr_data1,te_data1 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=100)
    while(i<4):
        modelmlp = classify(tr_data1, y_train,method='MLP',
                        h1=4000,activationfn=str[i])
    
    
        labels_predicted = test(test=te_data1,test_label=y_val,model=modelmlp,method='MLP')
        f_score, accuracy = evaluate(y_val, labels_predicted)
     
    
        print('Val - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
        acc.append(accuracy)
        actfn_index_arr.append(i)     
        
        
        i+=1
        
    plt.plot(actfn_index_arr, acc)
    plt.xlabel('Different activation functions that are used') 
    plt.ylabel('Accuracy') 
    plt.show()
    print(modelmlp.n_iter_)
    print(modelmlp.n_layers_)
    print(modelmlp.n_outputs_)
    print(modelmlp.out_activation_) 
    

In [22]:
def vary_num_pca_mlp(): #using mlp
    num_comp_pca=[]
    acc=[]
    i=50
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
   # trn_data, tst_data = image_prep(trn_data, tst_data)#, list(map(image_prep, tst_data))
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    trprep_data, teprep_data = image_prep(X_train, X_val)
    while(i<360):
        tr_data1,te_data1 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=i)
        modelmlp = classify(tr_data1, y_train,method='MLP',
                        h1=4000,activationfn='relu')
    
    
        labels_predicted = test(test=te_data1,test_label=y_val,model=modelmlp,method='MLP')
        f_score, accuracy = evaluate(y_val, labels_predicted)
     
        print('Val - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
        acc.append(accuracy)
        num_comp_pca.append(i)
        i+=50
    
    plt.plot(num_comp_pca, acc)
    plt.xlabel('No. of pca components used in MLP') 
    plt.ylabel('Accuracy') 
    plt.show()
    
    
    
    

## Main call for classifying 3 data types and Parameter tuning for Logistic Regression

In [23]:
def main_lr():
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    #trn_data, tst_data = image_prep(trn_data, tst_data)#, list(map(image_prep, tst_data))
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    ''' perform dimesioality reduction/feature extraction and classify the features into one of 10 classses
        print accuracy and f1-score.
    https://www.geeksforgeeks.org/args-kwargs-python/ #kwargs usage
        '''
    trprep_data, teprep_data = image_prep(X_train, X_val) 
        
    tr_data1,te_data1 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=100)
    tr_data2,te_data2 = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='lda',num_comp=350)
    modellr = classify(trprep_data,y_train,method='LogisticRegression',penalty='l2',num_iter=150,c=1.0)
      
    labels_predicted = test(test=teprep_data,test_label=y_val,model=modellr,method='LogisticRegression')
    f_score, accuracy = evaluate(y_val, labels_predicted)
    print('Val - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
    
    modellr = classify(tr_data1,y_train,method='LogisticRegression',penalty='l2',num_iter=150,c=1.0)
      
    labels_predicted = test(test=te_data1,test_label=y_val,model=modellr,method='LogisticRegression')
    f_score, accuracy = evaluate(y_val, labels_predicted)
    print('Val - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
    
    modellr = classify(tr_data2,y_train,test_label=y_val,method='LogisticRegression',penalty='l2',num_iter=150,c=1.0)
      
    labels_predicted = test(test=te_data2,test_label=y_val,model=modellr,method='LogisticRegression')
    f_score, accuracy = evaluate(y_val, labels_predicted)
    print('Val - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
    

In [24]:
def vary_c_lr():
    c_val_arr=[]
    acc=[]
    i=1.0
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    trprep_data, teprep_data = image_prep(X_train, X_val)
    #trprep_data,teprep_data = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=100)
    while(i<15):
        modellr = classify(trprep_data,y_train,method='LogisticRegression',penalty='l2',num_iter=150,c=i)
      
        labels_predicted = test(test=teprep_data,test_label=y_val,model=modellr,method='LogisticRegression')
        f_score, accuracy = evaluate(y_val, labels_predicted)
        print('Val - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
        acc.append(accuracy)
        c_val_arr.append(i) 
        i+=1
    plt.plot(c_val_arr, acc)
    plt.xlabel('C values (LR) ')
    plt.show()
#vary_c_lr()      

In [25]:
def vary_num_iter_lr():
    iter_val_arr=[]
    acc=[]
    i=150
    trn_data, trn_labels, tst_data, tst_labels = load_cifar()
    
    X_train, X_val, y_train, y_val = train_test_split(trn_data, trn_labels,test_size = 0.20) 
    trprep_data, teprep_data = image_prep(X_train, X_val)
    trprep_data,teprep_data = reduce_dim(train_data=trprep_data,test_data=teprep_data,train_label=y_train,method='pca',num_comp=100)
    while(i<5159):
        modellr = classify(trprep_data,y_train,method='LogisticRegression',penalty='l2',num_iter=i,c=1.0)
      
        labels_predicted = test(test=teprep_data,test_label=y_val,model=modellr,method='LogisticRegression')
        f_score, accuracy = evaluate(y_val, labels_predicted)
        print('Val - F1 score: {}\n Accuracy: {}'.format(f_score, accuracy))
        acc.append(accuracy)
        iter_val_arr.append(i) 
        i+=500
    plt.plot(iter_val_arr, acc)
    plt.xlabel('Number of iterationsin LR')
    plt.show()
#vary_num_iter_lr()       

## Caller of all functions

In [26]:
if __name__ == '__main__':
    
    
   
    #vary_num_trees()
    #vary_min_split_tree()
    #vary_depth_tree()
    #vary_num_pca()
    #vary_num_lda()
    
    #vary_num_pca_mlp()
    #num_neurons_l1()
    #vary_activ_fn()
    
    #main_lr()
    #vary_c_lr()
    #vary_num_iter_lr()
    
    #vary_pca_svm()
    #vary_num_iter_svm()
    #vary_ker_svm()
    
    #main_svm()
    
    #main_tree()
    #main_mlp()
    

SyntaxError: unexpected EOF while parsing (<ipython-input-26-018f62993263>, line 27)